**AIM 5003 Numerical Methods - Spring 2021**

**Project 2 - GPS Positioning**

**Aishwarya Singh**

# Introduction:

This project is a study of how the Global Positioning System (GPS) works and a simulation of a GPS receiver, along with an analysis of the system's sensitivity to measurement errors.

GPS consists of two basic components: satellites and a receiver. The satellites contain highly accurate atomic clocks and broadcast their position and the time according to their clock. The receiver also contains a (normal) clock and compares its time to the received times to get the travel time of the radio waves from each satellite.

Using at least three satellites the receiver is thus able to triangulate its own position _(x,y,z)_. The clock on the receiver is not atomic however and is thus less accurate than the satellite clocks, leading to the necessity of an additional factor, the time drift _d_. A fourth satellite is required to solve for receiver drift in addition to receiver position.

# Methodology:

#### Part 1:
The method we use for having our receiver determine its own position and drift _(x,y,z,d)_ is the Multivariate Newton method, given 4 satellites' positions and signal travel times as inputs. The solution is defined as an intersection point which satisfies:

$ \sqrt{(x − A_i)^2 + (y − B_i)^2 + (z − C_i)^2} - c(t_i − d) = 0 $

for all satellites (_i_ = 1 ... 4) simultaneously. _c_ = 299,792.458 km/s is the constant speed of light.

From the above we see that:

$ (x − A_i)^2 + (y − B_i)^2 + (z − C_i)^2 = (c(t_i − d))^2 $

so we define:

$ f(x, y, z, d) = (x − A_i)^2 + (y − B_i)^2 + (z − C_i)^2 - (c(t_i − d))^2 = 0 $

as the root we are to seek.

The Multivariate Newton method is a technique for finding the roots of nonlinear real-valued functions wherein we take an initial guess value for the input vector and iteratively update it by a correction factor obtained by dividing the function of the current input by its derivative function using the same input. It depends on the function being differentiable and the derivative being non-zero. In one dimension this updating rule is defined as:

$ x_{n+1} = x_n - \frac{f(x_n)}{f^\prime(x_n)} $

In the multivariate case, the derivative takes the form of a Jacobian matrix _J_, and division takes the form of multiplication by a matrix inverse. It is defined as:

$ x_{n+1} = x_n - J(x_n)^{-1}f(x_n) $

#### Parts 2 & 3:
In addition to solving for the receiver's position we also explore the conditioning of the system, first when choosing arbitrarily spread satellites and then while choosing very tightly clustered satellites. The atomic clocks aboard the satellites are accurate to 10 nanosecond intervals, so we introduce timing errors of 10 nanoseconds to each of our satellite time signals and analyze the positioning errors we see in each case.

The position errors for the receiver are defined as the maximum norm of the error in each spatial dimension:

$ \|(\Delta x, \Delta y, \Delta z)\|_{\infty} $

A physically dimensionless Error Magnification Factor for the system is defined as:

$ EMF = \frac{\|(\Delta x, \Delta y, \Delta z)\|_{\infty}}{c\|(\Delta t_1, ..., \Delta t_4)\|_{\infty}} $

The Condition Number for the system is defined as the maximum EMF attained across all combinations of 10 nanosecond errors for the 4 satellites. A larger Condition Number implies a more ill-conditioned system, and thus a less accurate GPS.

# Computer Experiments & Results:

In [ ]:
import numpy as np
import math

c = 299792.458 # km/s

**Function Definitions:**

In [ ]:
# length-n vector-valued function 'f' for n satellites; n = 4
# f = 0 (zero vector) satisifies intersection - 1 receiver, 4 satellites
# receiver is a length-4 np array with x, y, z, d
# satellites is a (n, 4) 2-d np array with A, B, C, t
def f(receiver, satellites):
    x = receiver[0]
    y = receiver[1]
    z = receiver[2]
    cd = receiver[3] # time values converted to distances
    
    f = np.zeros(len(satellites[:,0]))
    
    for i in range(len(f)):
        A = satellites[i,0]
        B = satellites[i,1]
        C = satellites[i,2]
        ct = satellites[i,3] # time values converted to distances
        
        f[i] = (x - A)**2 + (y - B)**2 + (z - C)**2 - (ct - cd)**2
        
    
    return f

# Jacobian function returns (n, 4) 2-d np array for n satellites; n = 4
# Each row is the gradient of 'f' for 1 satellite
def J(receiver, satellites):
    x = receiver[0]
    y = receiver[1]
    z = receiver[2]
    cd = receiver[3] # time values converted to distances
    
    J = np.zeros((len(satellites[:,0]), 4))
    
    for i in range(len(J[:,0])):
        A = satellites[i,0]
        B = satellites[i,1]
        C = satellites[i,2]
        ct = satellites[i,3] # time values converted to distances
        
        J[i,0] = 2*(x - A) # df/dx
        J[i,1] = 2*(y - B) # df/dy
        J[i,2] = 2*(z - C) # df/dz
        J[i,3] = 2*(ct - cd) # df/dcd
    
    return J

# Function to determine position and drift of receiver via the multivariate Newton method
def GetReceiver(satellites, guess, iter = 10):
    invector = guess
    
    for i in range(iter):
        F = f(invector, satellites)
        Fprime = J(invector, satellites)
        
        delta = np.dot(np.linalg.inv(Fprime), F)
        invector = invector - delta
    
    return invector


## Question 1:

In [ ]:
# speed of light is incorporated in satellite matrix for transmission delays
# matrix is now entirely in units of km
satellites = np.array([[15600, 7540, 20140, 0.07074*c],
                       [18760, 2750, 18610, 0.07220*c],
                       [17610, 14630, 13480, 0.07690*c],
                       [19170, 610, 18390, 0.07242*c]])

# initial guess for multivariate Newton method
guess = np.array([0, 0, 6370, 0])

soln = GetReceiver(satellites, guess, 100)

### Question 1 Results:

In [ ]:
print("Receiver Position (km):")
print("x =", soln[0])
print("y =", soln[1])
print("z =", soln[2], "\n")
print("Time Correction (s):")
print("d =", soln[3]/c)

Receiver Position (km):
x = -41.772709570847084
y = -16.78919410652623
z = 6370.059559223352 

Time Correction (s):
d = -0.0032015658295941516


## Question 2:

In [ ]:
rho = 26570 # km
d = 0.0001 # sec

# Arbitrary values:
phi = np.array((0, math.pi/6, math.pi/3, math.pi/2))
theta = np.array((0, (2/3)*math.pi, (4/3)*math.pi, 2*math.pi))

satellites = np.zeros((4,4))

for i in range(4):
    satellites[i,0] = rho * math.cos(phi[i]) * math.cos(theta[i])
    satellites[i,1] = rho * math.cos(phi[i]) * math.sin(theta[i])
    satellites[i,2] = rho * math.sin(phi[i])

receiver = np.array([0, 0, 6370, 0])

rangesqr = f(receiver, satellites) # this is (x-A)^2 + (y-B)^2 + (z-c)^2
Ri = np.array([math.sqrt(i) for i in rangesqr])

ti = d + (Ri/c)

satellites[:,3] = ti*c
receiver = np.array([0, 0, 6370, d*c])

# f(receiver, satellites) # check to see if this is ~0
# GetReceiver(satellites, guess, 100) # check to see if this is ~same as receiver

In [ ]:
print("Ranges for Arbitrary Satellites (km):")
print(int(Ri[0]), ',', int(Ri[1]), ',', int(Ri[2]), ',', int(Ri[3]),"\n")
print("Travel Times for Arbitrary Satellites (s):")
print(ti[0], ',', ti[1], ',', ti[2], ',', ti[3])

Ranges for Arbitrary Satellites (km):
27322 , 24026 , 21292 , 20200 

Travel Times for Arbitrary Satellites (s):
0.09123944127634147 , 0.08024504053553431 , 0.07112570824748687 , 0.06747994723002672


**Code for introducing timing errors on the order of 10 nanoseconds (10^-8 s):**

In [ ]:
delta = 10**(-8)
sign = (-1,1)

signs = []

for i in sign:
    for j in sign:
        for k in sign:
            for l in sign:
                signs.append([i,j,k,l])

signs = np.array(signs)
signs = np.delete(signs, (0,15), 0)

changes = np.zeros(np.shape(signs))

for i in range(len(signs[:,0])):
    changes[i] = (delta*signs[i]) + ti


In [ ]:
currentsat = satellites
newreceivers = np.zeros((len(changes[:,0]), 4))
changeinpos = np.zeros(len(changes[:,0]))

for i in range(len(changes[:,0])):
    currentsat[:,3] = (changes[i])*c
    newreceivers[i] = GetReceiver(currentsat, guess, 100)
    
    delx = receiver[0] - newreceivers[i,0]
    dely = receiver[1] - newreceivers[i,1]
    delz = receiver[2] - newreceivers[i,2]
    changeinpos[i] = np.linalg.norm((delx, dely, delz), np.inf)

emf = changeinpos/(c*delta) # since inf norm of all +/- 10^8 values is just 10^8

### Question 2 Results:

In [ ]:
print("Position Errors for all cases (km):")
print(changeinpos,"\n")
print("Error Magnification Factors for all cases:")
print(emf,"\n")
print("Maximum Position Error (m):")
print(1000*np.max(changeinpos),"\n")
print("Condition Number:")
print(np.max(emf))

Position Errors for all cases (km):
[0.00932976 0.00647257 0.00526109 0.00336124 0.00706555 0.00852064
 0.00285551 0.00285551 0.00852064 0.00706554 0.00336124 0.00526109
 0.00647257 0.00932975] 

Error Magnification Factors for all cases:
[3.11207241 2.15901837 1.75491022 1.12118964 2.35681235 2.84218024
 0.95249539 0.95249542 2.84218117 2.35681155 1.12118961 1.75490993
 2.15901858 3.11207091] 

Maximum Position Error (m):
9.329758376225072 

Condition Number:
3.112072411182897


## Question 3:

In [ ]:
# New values for tightly clustered satellites:
phi = np.array((math.pi/4, 1.015*math.pi/4, 1.030*math.pi/4, 1.045*math.pi/4))
theta = np.array((math.pi, 1.015*math.pi, 1.030*math.pi, 1.045*math.pi))

satellites = np.zeros((4,4))

for i in range(4):
    satellites[i,0] = rho * math.cos(phi[i]) * math.cos(theta[i])
    satellites[i,1] = rho * math.cos(phi[i]) * math.sin(theta[i])
    satellites[i,2] = rho * math.sin(phi[i])

receiver = np.array([0, 0, 6370, 0])

rangesqr = f(receiver, satellites) # this is (x-A)^2 + (y-B)^2 + (z-c)^2
Ri = np.array([math.sqrt(i) for i in rangesqr])

ti = d + (Ri/c)

satellites[:,3] = ti*c
receiver = np.array([0, 0, 6370, d*c])

# f(receiver, satellites) # check to see if this is ~0
# GetReceiver(satellites, guess, 100) # check to see if this is ~same as receiver

In [ ]:
print("Ranges for Tightly-Clustered Satellites (km):")
print(int(Ri[0]), ',', int(Ri[1]), ',', int(Ri[2]), ',', int(Ri[3]),"\n")
print("Travel Times for Tightly-Clustered Satellites (s):")
print(ti[0], ',', ti[1], ',', ti[2], ',', ti[3])

Ranges for Tightly-Clustered Satellites (km):
22520 , 22458 , 22396 , 22335 

Travel Times for Tightly-Clustered Satellites (s):
0.07522118789913924 , 0.0750133050029992 , 0.07480734646464027 , 0.0746033569459735


In [ ]:
for i in range(len(signs[:,0])):
    changes[i] = (delta*signs[i]) + ti

currentsat = satellites

newreceivers = np.zeros((len(changes[:,0]), 4))
changeinpos = np.zeros(len(changes[:,0]))

for i in range(len(changes[:,0])):
    currentsat[:,3] = (changes[i])*c
    newreceivers[i] = GetReceiver(currentsat, guess, 100)
    
    delx = receiver[0] - newreceivers[i,0]
    dely = receiver[1] - newreceivers[i,1]
    delz = receiver[2] - newreceivers[i,2]
    changeinpos[i] = np.linalg.norm((delx, dely, delz), np.inf)

emf = changeinpos/(c*delta) # since inf norm of all +/- 10^8 values is just 10^8

### Question 3 Results:

In [ ]:
print("Position Errors for all cases (km):")
print(changeinpos,"\n")
print("Error Magnification Factors for all cases:")
print(emf,"\n")
print("Maximum Position Error (m):")
print(1000*np.max(changeinpos),"\n")
print("Condition Number:")
print(np.max(emf))

Position Errors for all cases (km):
[194.99650454 571.24416179 381.44957891 584.62439941 785.32026923
   2.64434716 192.34158105 190.56866261   2.64394087 756.79848017
 568.62772639 388.5635931  587.30303003 193.20278164] 

Error Magnification Factors for all cases:
[ 65043.8326035  190546.5419643  127237.88365258 195009.70881833
 261954.64504506    882.05926697  64158.24545054  63566.86351508
    881.92374512 252440.80028604 189673.79305682 129610.86336004
 195903.20382074  64445.51104806] 

Maximum Position Error (m):
785320.2692257746 

Condition Number:
261954.6450450647


# Conclusions:

In Question 1, we demonstrate the operation of a GPS receiver as it successfully locates itself on the surface of the Earth using information from the four given satellites and the Multivariate Newton method.

In Question 2 and 3, we see the effects of satellite clustering on the system's conditioning.

We see that when the satellites are spread out in the sky (Question 2), the system is relatively well-conditioned. The positioning errors are small, on the order of a few meters. Thus the GPS is useful in this scenario.

We see that when the satellites are all clustered however in the same small region of the sky (Question 3), the system becomes ill-conditioned. The positioning errors are on the order of several hundred kilometers, and the Condition Number is many orders of magnitude larger than it was in the previous case. Thus we can see that the GPS is not very useful in this scenario, where even the minuscule timing errors of the satellites' highly-accurate atomic clocks result in the receiver being unable to locate itself.

# References:

1. Sauer, T. (2017). 4.5 _Nonlinear Least Squares_. In _Numerical Analysis_. Pearson.

2. _Multidimensional-Newton_. http://web.mit.edu/18.06/www/Spring17/Multidimensional-Newton.pdf

# Appendix:

All work done by Aishwarya Singh.